# LightFM runbook

## Loading libraries and setting parameters

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys

sys.path.append("../../..")

from src.common import helpers, config
from src.models.lightfm import LFM
from src.evaluation.evaluator import Evaluator

In [2]:
MODEL_NAME = "lightfm"

paths = config.Paths(
    dataset_name="jobs_published",
    target_users_name="subset_validation",
    model_name=MODEL_NAME,
)

In [3]:
no_components=30
k=5
n=10
learning_schedule="adagrad"
loss="logistic"
learning_rate=0.05
rho=0.95
epsilon=1e-06
item_alpha=0.0
user_alpha=0.0
max_sampled=10
random_state=42
epochs=20

N_RECOMMENDATIONS = 10

## Model training

In [4]:
model = LFM(
    no_components=no_components,
    k=k,
    n=n,
    learning_schedule=learning_schedule,
    loss=loss,
    learning_rate=learning_rate,
    rho=rho,
    epsilon=epsilon,
    item_alpha=item_alpha,
    user_alpha=user_alpha,
    max_sampled=max_sampled,
    random_state=random_state,
    epochs=epochs,
)

model.load_interactions(paths.train)

model.preprocess()

model.fit()

Epoch: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]


## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS, show_progress=True
)

model.save_recommendations(recommendations, paths.recommendations)

30000it [03:31, 141.83it/s]


## Evaluation

In [6]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=[MODEL_NAME],
)

evaluator.prepare()

evaluator.evaluate_models()

30000it [00:02, 14999.53it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
lightfm,0.001667,0.005902,0.002266,0.001834,0.003906,0.00217,0.005673,0.502665,0.0155,1.0,0.001031,2.333811,0.99948,0.0,0.0
